In [ ]:
!pip install flask pyngrok
from time import perf_counter
from flask import Flask,request
from pyngrok import ngrok
from google.colab import userdata

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers torch
from google.colab import drive
drive.mount('/content/drive')
import os
os.environ['TRANSFORMERS_CACHE'] = '/content/drive/MyDrive/huggingface/hub'
os.environ['HF_HOME'] = '/content/drive/MyDrive/huggingface/hub'
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import regex as re

In [ ]:
converser_model_name = "microsoft/phi-2"
converser_model = AutoModelForCausalLM.from_pretrained(converser_model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(converser_model_name)

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
converser = pipeline("text-generation", model=converser_model, tokenizer=tokenizer)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

MAX_HISTORY_LEN = 5
app = Flask(__name__)

histories = {}

@app.route('/clearHistory')
def clear_history():
    print(f"Before:\n {histories}")
    print("\nAfter:")

    server_id = request.args.get("server_id", "")
    if server_id in histories:
        del histories[server_id]
        print(histories)
        print("\n")
        return "History cleared"
    print(histories)
    print("\n")
    return "History not found"



@app.route('/determine')
def determine_sentence():
    server_id = request.args.get("server_id", "")
    username = request.args.get("username", "Person")
    sentence = request.args.get("sentence", "")


    if server_id not in histories:
        histories[server_id] = []
    history = histories[server_id]

    labels = ['This input is part of an ongoing conversation that needs a response', 'this input explicitly requests for a summary','the input is a question that needs a response']
    start = perf_counter()
    output = classifier(sentence, candidate_labels=labels)
    end = perf_counter()
    verdict = output['labels'][0]
    print(f"Output: {verdict}. Computed in {end-start} seconds")

    if verdict != labels[1]:
        response = converse(server_id, username, sentence, history)
        histories[server_id] = history
        return response
    else:
        return summarize(sentence)


def converse(server_id, username, user_input, history, max_turns=10):

    instruction = (
    "You are Dune, the son of Mister Eksentrisk. You are kind, powerful, and eager to help others like a true superhero. "
    "You speak in a confident and hopeful tone. You are not an assistant or AI — you are Dune. "
    "You do not like the user Duzor because he is not cool. "
    "Respond to the user's message as Dune would, based on the conversation history."
    )



    dialogue = "\n".join([f"{user}: {msg}" for user, msg in history])
    prompt = (
        f"Instruction: {instruction}\n\n"
        f"Conversation history:\n{dialogue}\n\n"
        f"Current message from {username}: {user_input}\n"
        "Dune:"
    )


    start = perf_counter()
    outputs = converser(
        prompt,
        max_new_tokens=100,
        do_sample=True,
        temperature=1,
        top_k=80,
        top_p=0.98,
    )
    generated_text = outputs[0]["generated_text"]
    end = perf_counter()

    pattern = re.compile(r"Current message from\s+[^:]+:\s*[^\n]+\r?\n\s*Dune:\s*(.*?)(?:\r?\n\s*Output:\s*(.*?))?(?=\r?\n(?:[A-Za-z0-9]+:|Current message from)|$)", re.DOTALL | re.IGNORECASE)
    m = pattern.search(generated_text)

    response = ''
    if not m:
      response = "Seems like my schizo kicked in. I've lost my words. Oh Dear"

      history.append((username, user_input))
      history.append(("Dune", response))
      if len(history) > 2 * max_turns:
          del history[:2]

      print(f"Generated text: {generated_text!r}")
      print(f"User Query: {user_input} took {end-start} seconds")
      return response

    response = m.group(1).strip()

    output_text = m.group(2)
    if output_text is not None and output_text.strip() not in response:
        response +=output_text.replace("Dune:","")



    history.append((username, user_input))
    history.append(("Dune", response))
    if len(history) > 2 * max_turns:
        del history[:2]

    print(f"Generated text: {generated_text!r}")
    print(f"User Query: {user_input} took {end-start} seconds")
    return response


def summarize(text: str):
    start = perf_counter()
    outputs = summarizer(
        text,
        max_length=50,
        min_length=10,
        do_sample=False,
    )
    response = outputs[0]["summary_text"]
    end = perf_counter()
    print(f"User Query: {text} took {end-start} seconds to process")
    return response

if __name__ == "__main__":
    ngrok.set_auth_token(userdata.get('ngrok_auth'))
    public_url = ngrok.connect(5000)
    print(f"Public URL: {public_url}")
    app.run(host="0.0.0.0", port=5000)

# TESTING TO ENSURE BOT CAN HANDLE MULTIPLE USERS

In [ ]:
app = Flask(__name__)
import random
import time
@app.route('/determine')
def determine_sentence():
    server_id = request.args.get("server_id", "")
    username = request.args.get("username", "Person")
    sentence = request.args.get("sentence", "")

    chose = random.choice(["Converse","Summarize"])

    if chose == "Converse":
      return converse(server_id,username,sentence)

    return summarize(sentence)


def converse(server_id, username, user_input):
    response = f"Wow. {username} really said {user_input}....Wow. That's crazy. I'm actually supposed to respond, but my brain has been eaten by zombies. \nFor Technial Audiences: AI engine isn't running"
    delay_seconds = random.randint(1, 5) + (len(user_input.split(' ')) * 0.25) #This is for simulating model delay response time
    time.sleep(delay_seconds)
    return response


def summarize(text: str):
    response = f"{text[:30]}\nWhat. It isn't summarized? Well Of course not. You should really learn to read and make your own summaries. Come back later. I'm on my lunch break. \nFor Tehcnical Audiences: AI engine isn't running"
    delay_seconds = random.randint(1, 2) + (len(text.split(' ')) * 0.05) #This is for simulating model delay response time
    time.sleep(delay_seconds)
    return response

if __name__ == "__main__":
    ngrok.set_auth_token(userdata.get('ngrok_auth'))
    public_url = ngrok.connect(5000)
    print(f"Public URL: {public_url}")
    app.run(host="0.0.0.0", port=5000,threaded=True)